In [ ]:
"""
    Utils functions for LSTM network.
"""

from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
import io
import numpy as np


def create_sequences(text, sequence_length, step):
    sequences = []
    next_chars = []
    for i in range(0, len(text) - sequence_length, step):
        sequences.append(text[i: i + sequence_length])
        next_chars.append(text[i + sequence_length])
    return sequences, next_chars


def build_model(sequence_length, chars):
    model = Sequential()
    model.add(LSTM(128, input_shape=(sequence_length, len(chars))))
    model.add(Dense(len(chars)))
    model.add(Activation('softmax'))

    optimizer = RMSprop(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)
    return model


def sample(preds, temperature=1.0):

    if temperature == 0:
        temperature = 1

    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def extract_characters(text):
    return sorted(list(set(text)))


def get_chars_index_dicts(chars):
    return dict((c, i) for i, c in enumerate(chars)), dict((i, c) for i, c in enumerate(chars))


def read_corpus(path):
    with io.open(path, 'r', encoding='utf8') as f:
        return f.read().lower()


def vectorize(sequences, sequence_length, chars, char_to_index, next_chars):
    X = np.zeros((len(sequences), sequence_length, len(chars)), dtype=np.bool)
    y = np.zeros((len(sequences), len(chars)), dtype=np.bool)
    for i, sentence in enumerate(sequences):
        for t, char in enumerate(sentence):
            X[i, t, char_to_index[char]] = 1
        y[i, char_to_index[next_chars[i]]] = 1

    return X, y


In [ ]:
from __future__ import print_function
import numpy as np
import random
import sys
from keras.models import load_model

SEQUENCE_LENGTH = 40
SEQUENCE_STEP = 3
PATH_TO_CORPUS = "jazz.txt"
EPOCHS10 = 10
DIVERSITY = 1.0

#Read the corpus and get unique characters from the corpus.
text = read_corpus(PATH_TO_CORPUS)
chars = extract_characters(text)

#Create sequences that will be used as the input to the network.
#Create next_chars array that will serve as the labels during the training.
sequences, next_chars = create_sequences(text, SEQUENCE_LENGTH, SEQUENCE_STEP)
char_to_index, indices_char = get_chars_index_dicts(chars)

#The network is not able to work with characters and strings, we need to vectorise.
X, y = vectorize(sequences, SEQUENCE_LENGTH, chars, char_to_index, next_chars)

#Define the structure of the model.
model = build_model(SEQUENCE_LENGTH, chars)

#Train the model

model.fit(X, y, batch_size=128, epochs=EPOCHS10)
#model = load_model("final.h5")  # you can skip training by loading the trained weights

#Pick a random sequence and make the network continue
for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    # insert your 40-chars long string. OBS it needs to be exactly 40 chars!
    sentence = "You are my light and my hope, Lord, You "
    sentence = sentence.lower() #stop here-------------^"
    generated += sentence

    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x = np.zeros((1, SEQUENCE_LENGTH, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_to_index[char]] = 1

        predictions = model.predict(x, verbose=0)[0]
        next_index = sample(predictions, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()